In [2]:
import sys
import json
import os
import traceback

def extract_printable_strings(path, minlen=4):
    """Fallback: pull readable ASCII sequences from a binary file."""
    out = []
    with open(path, "rb") as f:
        buf = bytearray()
        while True:
            b = f.read(1)
            if not b:
                if len(buf) >= minlen:
                    out.append(buf.decode("ascii", errors="ignore"))
                break
            v = b[0]
            if 32 <= v < 127:
                buf.append(v)
            else:
                if len(buf) >= minlen:
                    out.append(buf.decode("ascii", errors="ignore"))
                buf = bytearray()
    return out

In [4]:
# Parameters
path = "/data/gardeux/PersData.kat"

if not os.path.exists(path):
    print(f"File not found: {path}", file=sys.stderr)
    sys.exit(1)

In [6]:
import pythonnet  # type: ignore
pythonnet.load()  # loads CoreCLR if present; no-op on pythonnet 2

RuntimeError: Failed to create a default .NET runtime, which would
                    have been "mono" on this system. Either install a
                    compatible runtime or configure it explicitly via
                    `set_runtime` or the `PYTHONNET_*` environment variables
                    (see set_runtime_from_env).

In [ ]:
    import clr  # provided by pythonnet

    # Try to bring in BinaryFormatter
    # On .NET Framework / Mono: in mscorlib; on .NET (Core) types exist in System.Runtime.Serialization.Formatters
    try:
        clr.AddReference("System.Runtime.Serialization.Formatters")
    except Exception:
        # It's okay if this fails; types might already be in mscorlib/loaded
        pass

    from System import String, DateTime, Decimal, Enum, Array, Byte, SByte, Int16, Int32, Int64, UInt16, UInt32, UInt64, Boolean, Double, Single, Guid
    from System.Collections import IDictionary, IEnumerable
    from System.Runtime.Serialization.Formatters.Binary import BinaryFormatter
    from System.IO import FileStream, FileMode, FileAccess
    from System.Reflection import BindingFlags
    from System.Runtime.CompilerServices import RuntimeHelpers

    # Some modern runtimes throw immediately when constructing or using BinaryFormatter.
    try:
        bf = BinaryFormatter()
    except Exception as e:
        raise RuntimeError(
            "BinaryFormatter is not available/enabled on this runtime. "
            "Use Mono with pythonnet<3, or a Windows .NET Framework machine."
        ) from e

    # Deserialize
    fs = FileStream(path, FileMode.Open, FileAccess.Read)
    try:
        root = bf.Deserialize(fs)
    finally:
        fs.Close()

    # Helpers to convert .NET object graph to Python
    seen_ids = set()  # track by RuntimeHelpers.GetHashCode(obj) to avoid cycles

    def is_primitive_like(o):
        return (
            o is None
            or isinstance(o, (str, int, float, bool))
        )

    def dotnet_is_primitive_or_simple(o):
        t = o.GetType()
        # primitives, strings, decimals, enums, DateTime, Guid
        if t.IsPrimitive:  # bool, int32, etc
            return True
        if isinstance(o, (String, Decimal, DateTime, Guid)):
            return True
        if t.IsEnum:
            return True
        return False

    def to_py(o, depth=0):
        if o is None:
            return None

        # Avoid infinite recursion with object identity (reference equality)
        try:
            oid = RuntimeHelpers.GetHashCode(o)
        except Exception:
            # For Python primitives, just skip id tracking
            oid = None

        if oid is not None:
            if oid in seen_ids:
                return None
            seen_ids.add(oid)

        # Fast path for obvious Python types bridged by pythonnet
        if isinstance(o, (str, int, float, bool)):
            return o

        # Simple .NET value-like types
        if dotnet_is_primitive_or_simple(o):
            # Enums -> name; DateTime -> iso; Decimal -> float (best effort)
            t = o.GetType()
            try:
                if t.IsEnum:
                    return str(o)  # "EnumType.Value"
                if isinstance(o, DateTime):
                    return o.ToString("o")  # ISO 8601
                if isinstance(o, Decimal):
                    return float(o)  # may lose precision, but JSON-friendly
                if isinstance(o, Guid):
                    return str(o)
            except Exception:
                pass
            # Fallback: ToString
            try:
                return str(o)
            except Exception:
                return None

        # Byte[] -> base64-ish hex for JSON readability (or list of ints)
        try:
            if isinstance(o, Array) and o.GetType().GetElementType() in (Byte, SByte):
                # Return list of ints to keep it JSON-native
                return [int(b) for b in o]
        except Exception:
            pass

        # IDictionary
        if isinstance(o, IDictionary):
            py = {}
            try:
                for key in o.Keys:
                    k = to_py(key, depth + 1)
                    v = to_py(o[key], depth + 1)
                    py[str(k)] = v
            except Exception:
                # Best-effort iteration
                for entry in o:
                    try:
                        k = to_py(entry.Key, depth + 1)
                        v = to_py(entry.Value, depth + 1)
                        py[str(k)] = v
                    except Exception:
                        continue
            return py

        # IEnumerable (but not string/bytes)
        if isinstance(o, IEnumerable) and not isinstance(o, String):
            lst = []
            it = o.GetEnumerator()
            try:
                while it.MoveNext():
                    lst.append(to_py(it.Current, depth + 1))
            except Exception:
                # fallback: foreach in Python
                try:
                    for item in o:
                        lst.append(to_py(item, depth + 1))
                except Exception:
                    pass
            return lst

        # Reflect fields & simple properties
        pyobj = {}

        # Include .NET type name for context
        try:
            pyobj["$type"] = str(o.GetType().FullName)
        except Exception:
            pass

        # Instance fields (public + nonpublic)
        try:
            flags = BindingFlags.Instance | BindingFlags.Public | BindingFlags.NonPublic
            fields = o.GetType().GetFields(flags)
            for f in fields:
                try:
                    pyobj[f.Name] = to_py(f.GetValue(o), depth + 1)
                except Exception:
                    continue
        except Exception:
            pass

        # Public, readable, parameterless properties
        try:
            props = o.GetType().GetProperties(BindingFlags.Instance | BindingFlags.Public)
            for p in props:
                try:
                    if p.CanRead and p.GetIndexParameters().Length == 0:
                        name = p.Name
                        # Skip ubiquitous noisy props if desired:
                        if name in ("SyncRoot", "IsSynchronized", "Count"):
                            continue
                        pyobj[name] = to_py(p.GetValue(o, None), depth + 1)
                except Exception:
                    continue
        except Exception:
            pass

        return pyobj

    plain = to_py(root)
    json.dump(plain, sys.stdout, ensure_ascii=False, indent=2)
    return

except Exception as e:
    print(f"[!] Deserialization via pythonnet failed:\n{e}", file=sys.stderr)
    # Uncomment to debug:
    # traceback.print_exc()

# Fallback: extract printable strings so you can at least see likely field names
print("\n[!] Falling back to printable-strings extraction (deserialization unavailable).", file=sys.stderr)
strs = extract_printable_strings(path)
out = {
    "_note": "BinaryFormatter deserialization not available; showing printable strings only.",
    "file": os.path.abspath(path),
    "printable_strings": strs,
}
json.dump(out, sys.stdout, ensure_ascii=False, indent=2)
